##Importing Libraries

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier 
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [3]:
data=pd.read_csv("creditcard.csv")

In [4]:
Total_transactions = len(data)
normal = len(data[data.Class == 0])
fraudulent = len(data[data.Class == 1])
fraud_percentage = round(fraudulent/normal*100, 2)
print('Total number of Trnsactions are: {}'.format(Total_transactions))
print('Number of Normal Transactions are: {}'.format(normal))
print('Number of fraudulent Transactions are: {}'.format(fraudulent))
print('Percentage of fraud Transactions is: {}'.format(fraud_percentage))

Total number of Trnsactions are: 284807
Number of Normal Transactions are: 284315
Number of fraudulent Transactions are: 492
Percentage of fraud Transactions is: 0.17


We can say only 0.17% of transactions are fraudulent.

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [6]:
data.shape

(284807, 31)

In [7]:
data.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,1.168375e-15,3.416908e-16,-1.379537e-15,2.074095e-15,9.604066e-16,1.487313e-15,-5.556467e-16,1.213481e-16,-2.406331e-15,...,1.654067e-16,-3.568593e-16,2.578648e-16,4.473266e-15,5.340915e-16,1.683437e-15,-3.660091e-16,-1.227390e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [8]:
sc = StandardScaler()
amount = data['Amount'].values
data['Amount'] = sc.fit_transform(amount.reshape(-1, 1))

In [9]:
data.drop(['Time'], axis=1, inplace=True)
data.drop_duplicates(inplace=True)

In [10]:
data.shape

(275663, 30)

##Spliting data into training set and test set

In [11]:
X = data.drop('Class', axis = 1).values
y = data['Class'].values

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 1)

##Model Building

##Decision Tree


In [13]:
DT = DecisionTreeClassifier(max_depth = 4, criterion = 'entropy')
DT.fit(X_train, y_train)
dt_pred = DT.predict(X_test)

In [22]:
print('Accuracy score of the Decision Tree model is: {:.2f}%'.format(accuracy_score(y_test, dt_pred)*100))
confusion_matrix(y_test, dt_pred)

Accuracy score of the Decision Tree model is: 99.91%


array([[68770,    18],
       [   41,    87]])

##K-Nearest Neighbors

In [24]:
KNN = KNeighborsClassifier(n_neighbors = 7)
KNN.fit(X_train, y_train)
knn_pred = KNN.predict(X_test)

In [25]:
print('Accuracy score of the K-Nearest Neighbors model is: {:.2f}%'.format(accuracy_score(y_test, knn_pred)*100))
confusion_matrix(y_test, knn_pred)

Accuracy score of the K-Nearest Neighbors model is: 99.93%


array([[68772,    16],
       [   33,    95]])

##Logistic Regression

In [27]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)

In [28]:
print('Accuracy score of the Logistic Regression model is: {:.2f}%'.format(accuracy_score(y_test, lr_pred)*100))
confusion_matrix(y_test, lr_pred)

Accuracy score of the Logistic Regression model is: 99.90%


array([[68772,    16],
       [   56,    72]])

##Support Vector Machines

In [30]:
svm = SVC()
svm.fit(X_train, y_train)
svm_pred = svm.predict(X_test)

In [31]:
print('Accuracy score of the Support Vector Machines model is: {:.2f}%'.format(accuracy_score(y_test, svm_pred)*100))
confusion_matrix(y_test, svm_pred)

Accuracy score of the Support Vector Machines model is: 99.93%


array([[68785,     3],
       [   44,    84]])

##Random Forest

In [32]:
rf = RandomForestClassifier(max_depth = 4)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)

In [41]:
print('Accuracy score of the Random Forest model is: {:.2f}%'.format(accuracy_score(y_test, rf_pred)*100))
confusion_matrix(y_test, rf_pred)

Accuracy score of the Random Forest model is: 99.92%


array([[68778,    10],
       [   47,    81]])

##XGBoost

In [34]:
xgb = XGBClassifier(max_depth = 4)
xgb.fit(X_train, y_train)
xgb_pred = xgb.predict(X_test)

In [42]:
print('Accuracy score of the XGBoost model is: {:.2f}%'.format(accuracy_score(y_test, xgb_pred)*100))
confusion_matrix(y_test, xgb_pred)

Accuracy score of the XGBoost model is: 99.94%


array([[68784,     4],
       [   34,    94]])

##Catboost

In [36]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 7.7 MB/s eta 0:00:00


In [37]:
from catboost import CatBoostClassifier
cat_boost = CatBoostClassifier()
cat_boost.fit(X_train, y_train)
cat_pred = cat_boost.predict(X_test)

Learning rate set to 0.100375
0:	learn: 0.3569222	total: 214ms	remaining: 3m 33s
1:	learn: 0.1980382	total: 379ms	remaining: 3m 9s
2:	learn: 0.1070370	total: 489ms	remaining: 2m 42s
3:	learn: 0.0590893	total: 630ms	remaining: 2m 36s
4:	learn: 0.0352181	total: 737ms	remaining: 2m 26s
5:	learn: 0.0220055	total: 953ms	remaining: 2m 37s
6:	learn: 0.0145488	total: 1.2s	remaining: 2m 50s
7:	learn: 0.0104005	total: 1.32s	remaining: 2m 44s
8:	learn: 0.0079780	total: 1.48s	remaining: 2m 43s
9:	learn: 0.0063361	total: 1.72s	remaining: 2m 50s
10:	learn: 0.0052550	total: 1.92s	remaining: 2m 53s
11:	learn: 0.0045281	total: 2.12s	remaining: 2m 54s
12:	learn: 0.0039879	total: 2.29s	remaining: 2m 53s
13:	learn: 0.0036216	total: 2.5s	remaining: 2m 55s
14:	learn: 0.0033344	total: 2.74s	remaining: 2m 59s
15:	learn: 0.0031232	total: 2.98s	remaining: 3m 3s
16:	learn: 0.0029626	total: 3.29s	remaining: 3m 10s
17:	learn: 0.0028350	total: 3.93s	remaining: 3m 34s
18:	learn: 0.0027222	total: 4.71s	remaining: 4m 

In [40]:
print('Accuracy score of the XGBoost model is: {:.2f}%'.format(accuracy_score(y_test, cat_pred)*100))
confusion_matrix(y_test, cat_pred)

Accuracy score of the XGBoost model is: 99.95%


array([[68784,     4],
       [   31,    97]])

##K-fold cross validation

In [50]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = cat_boost, X = X_train, y = y_train, cv = 10) 
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Streaming output truncated to the last 5000 lines.
6:	learn: 0.0191204	total: 370ms	remaining: 52.5s
7:	learn: 0.0132394	total: 416ms	remaining: 51.6s
8:	learn: 0.0098740	total: 472ms	remaining: 52s
9:	learn: 0.0076332	total: 531ms	remaining: 52.5s
10:	learn: 0.0061264	total: 584ms	remaining: 52.5s
11:	learn: 0.0051968	total: 654ms	remaining: 53.8s
12:	learn: 0.0045276	total: 714ms	remaining: 54.2s
13:	learn: 0.0040931	total: 764ms	remaining: 53.8s
14:	learn: 0.0037243	total: 813ms	remaining: 53.4s
15:	learn: 0.0034632	total: 865ms	remaining: 53.2s
16:	learn: 0.0032806	total: 924ms	remaining: 53.4s
17:	learn: 0.0031203	total: 974ms	remaining: 53.1s
18:	learn: 0.0029877	total: 1.02s	remaining: 52.9s
19:	learn: 0.0029021	total: 1.08s	remaining: 53.2s
20:	learn: 0.0028321	total: 1.14s	remaining: 53.2s
21:	learn: 0.0027571	total: 1.19s	remaining: 53.1s
22:	learn: 0.0027152	total: 1.27s	remaining: 53.9s
23:	learn: 0.0026763	total: 1.32s	remaining: 53.6s
24:	learn: 0.0025988	total: 1.38s	rem

We got the best accuracy of 99.96% with CatBoost Model.